# Machine learning using EKF

In [6]:
from IPython import display
import os
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from load import load_abalone_data,load_bikes_data
# Importing Pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from  sklearn.datasets import make_regression
from sklearn.datasets import load_boston
from tqdm import tqdm 
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [7]:
# Select device which you are going to use for training
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


### Import Data Sets
Testing using a toy sine data

In [36]:
X,y = load_bikes_data()
print(y.shape)
print(X.shape)

(17379,)
(17379, 14)


## Data Partition


In [37]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size=.5)
X_val,X_test, y_val,y_test = train_test_split(X_test,y_test,test_size=0.5)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

# x_train_scaled = scaler.fit_transform(x_train)
# x_test_scaled = scaler.transform(x_test)

(8689, 14)
(4345, 14)
(4345, 14)


## Define Neural network

In [39]:
class MLP(nn.Module):
    def __init__(self, n_inputs, n_hidden_layer, n_outputs,bias=True):
        super(MLP, self).__init__()
        # YOUR CODE HERE
        #raise NotImplementedError()
        self.fc1 = nn.Linear(n_inputs, n_hidden_layer, bias)
        self.fc2 = nn.Linear(n_hidden_layer, n_hidden_layer, bias)
        #self.fc3 = nn.Linear(n_hidden_layer, n_hidden_layer, bias)
        self.fc4 = nn.Linear(n_hidden_layer, n_outputs, bias)
        

    def forward(self, x):
        # YOUR CODE HERE
        #raise NotImplementedError()
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        #x = torch.tanh(self.fc3(x))
        x = self.fc4(x)
        return x    

## Using EKF for learning 

In [40]:
  
def getWeights(net):
    weight_mat = []
    for name,param in net.named_parameters():
        if (len(list(param.data.shape)) == 2):
            weight_mat.append(param.data.flatten())
        
    weight_mat = torch.cat(weight_mat, dim=0)       
    return weight_mat.view(-1, 1)

def getWeightsgrad(net):
    weight_grad_mat = []
    for name,param in net.named_parameters():
        if (len(list(param.grad.shape)) == 2):
            weight_grad_mat.append(param.grad.flatten())
    weight_grad_mat = torch.cat(weight_grad_mat)       
    return weight_grad_mat.view(-1, 1)

def setWeights(net, weight_mat):
    mem_ind = 0;
    for name,param in net.named_parameters():
        if (len(list(param.data.shape)) == 2):
            param.data = weight_mat[mem_ind:mem_ind+torch.numel(param.data)].view(param.data.shape)
            mem_ind = torch.numel(param.data)
    


In [50]:
# Define number of Input and Output layers
torch.set_default_dtype(torch.float64)
n_inputs = X_train.shape[1]
n_outputs = 1
n_hidden_layer = 10

mlp_EKF = MLP(n_inputs,n_hidden_layer, n_outputs, bias = False)
mlp_EKF = mlp_EKF.to(device)
n_epochs = 1

# Define EKF covariances
weight_mat = getWeights(mlp_EKF).to(device)
print(f"Shape of W:{weight_mat.shape}")
# System Noise or also known as training  noise  
Q = 1e-16*torch.eye(weight_mat.shape[0],device=device, dtype=torch.float64)
# Measurement noise or noise in targets 
R = 10*torch.eye(n_outputs,device=device, dtype=torch.float64)
#Covariance Matrix
P = 100*torch.eye(weight_mat.shape[0],device=device, dtype=torch.float64)
print(f"Shape of P:{P.shape}")

print(f"network {mlp_EKF}")
print(weight_mat.shape)


def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)

mlp_EKF.apply(init_weights)
with torch.no_grad():
    print(mlp_EKF.fc1.weight.mean())
    print(mlp_EKF.fc1.weight.std())
init_weights = getWeights(mlp_EKF)

Shape of W:torch.Size([250, 1])
Shape of P:torch.Size([250, 250])
network MLP(
  (fc1): Linear(in_features=14, out_features=10, bias=False)
  (fc2): Linear(in_features=10, out_features=10, bias=False)
  (fc4): Linear(in_features=10, out_features=1, bias=False)
)
torch.Size([250, 1])
tensor(-0.0262)
tensor(0.2910)


## Torch Implementation

In [51]:
setWeights(mlp_EKF,init_weights)
weight_mat = getWeights(mlp_EKF).to(device)
Q = 1e-16*torch.eye(weight_mat.shape[0],device=device, dtype=torch.float64)
# Measurement noise or noise in targets 
R = 10*torch.eye(n_outputs,device=device, dtype=torch.float64)
#Covariance Matrix
P = 100*torch.eye(weight_mat.shape[0],device=device, dtype=torch.float64)

with torch.no_grad():
    print(f"weights mean:{mlp_EKF.fc1.weight.mean()}")
    print(f"weights std: {mlp_EKF.fc1.weight.std()}")
x_tensor = torch.tensor(X_train, device=device, dtype=torch.float64)
y_tensor = torch.tensor(y_train, device=device, dtype=torch.float64)
x_val_tensor = torch.tensor(X_val, device=device, dtype=torch.float64)
print_every = 500
n_epochs = 1
for epoch in range(n_epochs):
    
    outputs = [] 
#     rand_idx = np.random.permutation(len(X_train))
#     rand_x = X_train[rand_idx]
#     rand_y = y_train[rand_idx]
#     x_tensor = torch.tensor(rand_x, device=device, dtype=torch.float64)
#     y_tensor = torch.tensor(rand_y, device=device, dtype=torch.float64)
    #Calling Backward for each sample
    for i in (range(x_tensor.shape[0])):
        output = mlp_EKF(x_tensor[i])
        outputs.append(output)

        mlp_EKF.zero_grad()
        output.backward(torch.ones_like(output))
        #calculate loss
        loss = (y_tensor[i]-output).view(1,-1)

        H = getWeightsgrad(mlp_EKF).to(device).view(1,-1)
        #update weights using EKF filter Update
        intermediate = torch.mm(torch.mm(H, P), torch.t(H))

        Ak = torch.inverse(R + intermediate)

        Kk = torch.mm(torch.mm(P, torch.t(H)), Ak)

        weight_mat = weight_mat + torch.mm(Kk, loss)
        P = P + Q - torch.mm(torch.mm(Kk,H),P)
        setWeights(mlp_EKF,weight_mat)
        
        if(i%print_every ==0 ):#and i>100 and i<200):
            with torch.no_grad():
                y_pred = mlp_EKF.forward(x_val_tensor)
                y_pred = y_pred.cpu().data.numpy()
                error = mean_squared_error(y_val,y_pred)
                val_error =  np.sqrt(error)
            print(f"sample:{i} train error: {loss.item()} val error:{val_error.item()}")


weights mean:-0.02624640401302001
weights std: 0.29104142064254307
sample:0 train error: 152.10806333957743 val error:259.7315785393175
sample:500 train error: 342.1241352923023 val error:283.6231814666337
sample:1000 train error: 33.00583073380382 val error:281.2118857287086
sample:1500 train error: 75.07724597989258 val error:279.72236846515017
sample:2000 train error: 291.9850075191327 val error:278.88024249434466
sample:2500 train error: 321.0887648013807 val error:277.4267923094306
sample:3000 train error: 220.90891662836538 val error:277.2886861459389
sample:3500 train error: 275.08315439040695 val error:275.12974840179
sample:4000 train error: 258.1054780181637 val error:275.14640144014174
sample:4500 train error: 515.8006073800896 val error:318.85998582858247
sample:5000 train error: 99.31375537345293 val error:332.2523418780763
sample:5500 train error: 1002.2551294179297 val error:671.6113101313036
sample:6000 train error: 435.7608709648577 val error:382.9043923459573
sample:6

## Numpy intermediate

In [52]:
setWeights(mlp_EKF,init_weights)
weight_mat = getWeights(mlp_EKF).to(device)
Q = 1e-16*torch.eye(weight_mat.shape[0],device=device, dtype=torch.float64)
# Measurement noise or noise in targets 
R = 9*torch.eye(n_outputs,device=device, dtype=torch.float64)
#Covariance Matrix
P = 100*torch.eye(weight_mat.shape[0],device=device, dtype=torch.float64)

P = P.numpy()
Q = Q.numpy()
R = R.numpy()

with torch.no_grad():
    print(f"weights mean:{mlp_EKF.fc1.weight.mean()}")
    print(f"weights std: {mlp_EKF.fc1.weight.std()}")
x_tensor = torch.tensor(X_train, device=device, dtype=torch.float64)
y_tensor = torch.tensor(y_train, device=device, dtype=torch.float64)
x_val_tensor = torch.tensor(X_val, device=device, dtype=torch.float64)
print_every = 500
n_epochs =2
for epoch in range(n_epochs):
    
    outputs = [] 
    #Calling Backward for each sample
    for i in (range(x_tensor.shape[0])):
        output = mlp_EKF(x_tensor[i])
        outputs.append(output)

        mlp_EKF.zero_grad()
        output.backward(torch.ones_like(output))
        #calculate loss
        loss = (y_tensor[i]-output).view(1,-1)

        H = getWeightsgrad(mlp_EKF).to(device).view(1,-1)
        H = H.numpy()
        #update weights using EKF filter Update
        intermediate = H@P@H.T

        Ak = 1/(R + intermediate)

        Kk = P@H.T@Ak
        change = Kk @ loss.detach().numpy()
        change = torch.tensor(change,device=device, dtype=torch.float64)
        weight_mat = weight_mat + change
        P = P + Q - Kk@H@P
        setWeights(mlp_EKF,weight_mat)
        
        if(i%print_every ==0 ): #and i>100 and i<200):
            with torch.no_grad():
                y_pred = mlp_EKF.forward(x_val_tensor)
                y_pred = y_pred.cpu().data.numpy()
                error = mean_squared_error(y_val,y_pred)
                val_error =  np.sqrt(error)
            print(f"sample:{i} train error: {loss.item()} val error:{val_error.item()}")


weights mean:-0.02624640401302001
weights std: 0.29104142064254307
sample:0 train error: 152.10806333957743 val error:259.7297900225779
sample:500 train error: 694.1500547381894 val error:599.9638608404874
sample:1000 train error: 432.7998560970372 val error:642.6209637758285
sample:1500 train error: 501.01661695711044 val error:665.9116881841736
sample:2000 train error: 735.8732221578844 val error:682.1711112712586
sample:2500 train error: 482.22500303195784 val error:413.8684595004743
sample:3000 train error: 335.1044879933766 val error:372.0607093915069
sample:3500 train error: 373.2101863122872 val error:355.6031962234349
sample:4000 train error: 344.32170837207957 val error:345.375342342791
sample:4500 train error: 542.0262930083561 val error:338.29112847507014
sample:5000 train error: 100.44508523440648 val error:332.7340135442208
sample:5500 train error: 547.914883799384 val error:328.06367988581985
sample:6000 train error: 370.65361810488895 val error:324.5001745902667
sample:6

In [53]:
with torch.no_grad():
    x_test = torch.tensor(X_test, device=device, dtype=torch.float64)
    y_pred = mlp_EKF.forward(x_test)
    y_pred = y_pred.cpu().data.numpy()
    error = mean_squared_error(y_test,y_pred)
    print(np.sqrt(error))

291.83750948497635


In [15]:
H.dtype

dtype('float64')

In [13]:
R

array([[10.]])